In [1]:
import pandas as pd
import re

In [2]:
filepath="ecoli_cluster.tab"
cluster_df=pd.read_csv(filepath,delimiter='\t', dtype="object")
int_lst=["ClusterID", "HomClusterID","Size"]
cluster_df[int_lst]=cluster_df[int_lst].astype(int)
float_lst=["FuncCat(mbgd)", "FuncCat(cog)", "FuncCat(kegg)", "FuncCat(tigr)"]
cluster_df[float_lst]=cluster_df[float_lst].astype(float)
cluster_df.head()

,ClusterID,HomClusterID,Size,gm03458,gm03461,gm03462,gm03463,gm03464,gm03466,gm04123,...,elx,elc,eld,Gene,FuncCat(mbgd),FuncCat(cog),FuncCat(kegg),FuncCat(tigr),Description,Family
0,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.6,3.7,99.2,16.4,Hypothetical protein,Gene1
1,2,1,129,gm03458:BU34_RS13425,gm03461:CF59_RS01230,gm03462:CF57_RS21960,gm03463:CF60_RS01235,gm03464:CF61_RS22515,gm03466:CF58_RS01195,gm04123:TZ57_RS20530,...,elx:CDCO157_RS25725,elc:I14_RS22845,eld:I02_RS22840,uvrA,8.2,2.1,2.4,7.1,UvrABC system protein A,uvrA
2,3,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.3,1.1,3.1,11.1,Predicted in CGM,Gene3
3,4,1,93,gm03458:BU34_RS13270,gm03461:CF59_RS01385,gm03462:CF57_RS21805,gm03463:CF60_RS01390,gm03464:CF61_RS22360,gm03466:CF58_RS01350,gm04123:TZ57_RS20685,...,NaN,elc:I14_RS23035,eld:I02_RS23030,alsA,7.3,1.1,3.1,11.3,D-allose import ATP-binding protein AlsA,alsA
4,5,1,128,gm03458:BU34_RS18125,gm03461:CF59_RS13450,gm03462:CF57_RS09365,gm03463:CF60_RS13280,gm03464:CF61_RS09905,gm03466:CF58_RS13525,gm04123:TZ57_RS09800,...,elx:CDCO157_RS13180,elc:I14_RS10600,eld:I02_RS10595,araG,7.3,1.1,3.1,11.3,Arabinose import ATP-binding protein AraG,araG


In [4]:
species_lst=[s.strip() for s in open("species.list", 'r').readlines()]
species_lst

['ecg',
 'ese',
 'ecoj',
 'ecp',
 'eci',
 'ecz',
 'eab',
 'elc',
 'eln',
 'ecq',
 'eoc',
 'ecm',
 'gm03564']

In [6]:
species=species_lst[0]

filepath="/data/mitsuki/data/mbgd/gene/{}.gene".format(species)
gene_df=pd.read_csv(filepath, delimiter='\t', header=None)
gene_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,ecg,E2348C_0001,thrL,190,255,1,190..255,CDS,11,1,YP_002327593.1,215485162,7062056,thr operon leader peptide,21,chromosome-1-1
1,ecg,E2348C_0002,thrA,336,2798,1,336..2798,CDS,11,1,YP_002327594.1,215485163,7061796,bifunctional aspartokinase I/homoserine dehydr...,820,chromosome-1-1
2,ecg,E2348C_0003,thrB,2800,3732,1,2800..3732,CDS,11,1,YP_002327595.1,215485164,7061793,homoserine kinase,310,chromosome-1-1
3,ecg,E2348C_0004,thrC,3733,5019,1,3733..5019,CDS,11,1,YP_002327596.1,215485165,7061794,threonine synthase,428,chromosome-1-1
4,ecg,E2348C_0005,yaaX,5287,5529,1,5287..5529,CDS,11,1,YP_002327597.1,215485166,7061795,hypothetical protein,80,chromosome-1-1


In [ ]:
gene_lst=list(set(gene_df[1]))
gene2family_dct={}#map gene to set of family

#initialize dictionary with empty set
for gene in gene_lst:
    gene2family_dct[gene]=set()
    
#update corresponding set
pattern = r"([^()]+)(\([0-9]+\))?"
r=re.compile(pattern)
for _,row in cluster_df[["Family", species]].iterrows():
    if isinstance(row[species], str):
        for orfId in row[species].split():
            gene=r.findall(orfId)[0][0].split(':')[1]#drop (num) and genome name
            gene2family_dct[gene].update([row["Family"]])#outer [] to add as string           

#create lookup_df from gene2family_dct
dct_lst=[]
for gene in gene_lst:
    dct={}
    dct[1]=gene
    if len(gene2family_dct[gene])==0:
        dct["Family"]="-1"
    else:
        dct["Family"]=','.join(list(gene2family_dct[gene]))
    dct_lst.append(dct)
lookup_df=pd.DataFrame(dct_lst)

gene_df=pd.merge(gene_df, lookup_df, on=1, how="left")
gene_df.to_csv(filepath, index=False,header=None,sep='\t')